### Mill integration

In this specific example, we show, how `HierarchicalUtils.jl` can be used with `Mill.jl` library

In [5]:
using Pkg
pkg"activate"
using HierarchicalUtils
using Mill

 Activating environment at `~/.julia/dev/HierarchicalUtils/examples/Project.toml`


In [6]:
n1 = BagNode(BagNode(ArrayNode(randn(4,10)),[1:2,3:4,5:5,6:7,8:10]),[1:1,2:3,4:5])
n2 = ProductNode((ch1 = n1, ch2 = deepcopy(n1)))
ds = ProductNode((ch1 = n2, ch2 = deepcopy(n2), ch3 = deepcopy(n2)))

ProductNode
  ├── ch1: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          ⋮
  │          └── ch2: BagNode with 3 bag(s)
  │                     ⋮
  ├── ch2: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          ⋮
  │          └── ch2: BagNode with 3 bag(s)
  │                     ⋮
  └── ch3: ProductNode
             ├── ch1: BagNode with 3 bag(s)
             │          ⋮
             └── ch2: BagNode with 3 bag(s)
                        ⋮

In [8]:
printtree(ds)

ProductNode
  ├── ch1: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          └── BagNode with 5 bag(s)
  │          │                └── ArrayNode(4, 10)
  │          └── ch2: BagNode with 3 bag(s)
  │                     └── BagNode with 5 bag(s)
  │                           └── ArrayNode(4, 10)
  ├── ch2: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          └── BagNode with 5 bag(s)
  │          │                └── ArrayNode(4, 10)
  │          └── ch2: BagNode with 3 bag(s)
  │                     └── BagNode with 5 bag(s)
  │                           └── ArrayNode(4, 10)
  └── ch3: ProductNode
             ├── ch1: BagNode with 3 bag(s)
             │          └── BagNode with 5 bag(s)
             │                └── ArrayNode(4, 10)
             └── ch2: BagNode with 3 bag(s)
                        └── BagNode with 5 bag(s)
                              └── ArrayNode(4, 10)

In [9]:
printtree(ds; trunc=1)

ProductNode
  ⋮

In [10]:
printtree(ds; trunc=2)

ProductNode
  ├── ch1: ProductNode
  │          ⋮
  ├── ch2: ProductNode
  │          ⋮
  └── ch3: ProductNode
             ⋮

Default printing (overriding `Base.show`) uses `trunc = 3`

In [11]:
printtree(ds; trunc=3)

ProductNode
  ├── ch1: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          ⋮
  │          └── ch2: BagNode with 3 bag(s)
  │                     ⋮
  ├── ch2: ProductNode
  │          ├── ch1: BagNode with 3 bag(s)
  │          │          ⋮
  │          └── ch2: BagNode with 3 bag(s)
  │                     ⋮
  └── ch3: ProductNode
             ├── ch1: BagNode with 3 bag(s)
             │          ⋮
             └── ch2: BagNode with 3 bag(s)
                        ⋮

In [12]:
nnodes(ds)

22

In [13]:
nleafs(ds)

6

In [14]:
NodeIterator(ds) |> collect

22-element Array{AbstractNode{…},1}:
 ProductNode
 ProductNode
 BagNode
 BagNode
 ArrayNode
 BagNode
 BagNode
 ArrayNode
 ProductNode
 BagNode
 BagNode
 ArrayNode
 BagNode
 BagNode
 ArrayNode
 ProductNode
 BagNode
 BagNode
 ArrayNode
 BagNode
 BagNode
 ArrayNode

In [15]:
LeafIterator(ds) |> collect

6-element Array{ArrayNode{…},1}:
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode

In [16]:
TypeIterator(ds, BagNode) |> collect

12-element Array{BagNode{…},1}:
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode

In [17]:
pred(n::BagNode) = length(n.bags) > 4
pred(n::ProductNode) = :ch3 in keys(n.data)
pred(n) = false
PredicateIterator(ds, pred) |> collect

7-element Array{AbstractNode{…},1}:
 ProductNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode
 BagNode

In [18]:
pred2(n::ArrayNode) = size(n.data, 2) > 5
pred2(n) = false
PredicateIterator(ds, pred2) |> collect

6-element Array{ArrayNode{…},1}:
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode
 ArrayNode

In [19]:
m = reflectinmodel(ds)

ProductModel ↦ ArrayModel(Dense(30, 10))
  ├── ch1: ProductModel ↦ ArrayModel(Dense(20, 10))
  │          ├── ch1: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
  │          │          ⋮
  │          └── ch2: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
  │                     ⋮
  ├── ch2: ProductModel ↦ ArrayModel(Dense(20, 10))
  │          ├── ch1: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
  │          │          ⋮
  │          └── ch2: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
  │                     ⋮
  └── ch3: ProductModel ↦ ArrayModel(Dense(20, 10))
             ├── ch1: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
             │          ⋮
             └── ch2: BagModel ↦ SegmentedMean(10) ↦ ArrayModel(Dense(10, 10))
                        ⋮

In [20]:
NodeIterator((m, ds)) |> collect

22-element Array{Tuple{AbstractMillModel{…},AbstractNode{…}},1}:
 (ProductModel, ProductNode)
 (ProductModel, ProductNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)
 (ProductModel, ProductNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)
 (ProductModel, ProductNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)
 (BagModel, BagNode)
 (BagModel, BagNode)
 (ArrayModel, ArrayNode)